In [1]:
import json

In [2]:
import mysql.connector

In [3]:
# establish connection to database
cnx = mysql.connector.connect(user='root', 
                              password='123456', 
                              host='localhost',
                              database='mydatabase',
                              auth_plugin = 'mysql_native_password')

cursor = cnx.cursor()

# create tables if they do not already exist
cursor.execute(" CREATE TABLE IF NOT EXISTS user (\
  id VARCHAR(255) PRIMARY KEY,\
  name VARCHAR(255),\
  screen_name VARCHAR(255),\
  location VARCHAR(255),\
  description TEXT,\
  followers_count INT,\
  friends_count INT,\
  favourites_count INT,\
  statuses_count INT,\
  verified BOOLEAN,\
  created_at VARCHAR(255),\
  url VARCHAR(255) )")


cursor.execute("CREATE TABLE IF NOT EXISTS profile (\
  user_id VARCHAR(255),\
  profile_bg_color VARCHAR(255),\
  profile_bg_image_url_https VARCHAR(255),\
  profile_bg_file VARCHAR(255),\
  profile_side_bar_border_color VARCHAR(255),\
  profile_sidebar_fill_color VARCHAR(255),\
  profile_text_color VARCHAR(255),\
  profile_image_url_https VARCHAR(255),\
  profile_banner_url VARCHAR(255),\
  FOREIGN KEY (user_id) REFERENCES user(id) )") 


In [4]:
with open("corona-out-2", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            if (data['text'].startswith('RT')):
                #adding retweet in user details
                cursor.execute("SELECT * FROM user WHERE id = %s", (data['user']['id_str'],))
                if (cursor.fetchone() is None):
                    user = data['user']
                    user_id = user['id_str']
                    user_values = (user_id, user['name'], user['screen_name'], user['location'], user['description'], 
                     user['followers_count'], user['friends_count'], user['favourites_count'], user['statuses_count'], 
                     user['verified'], user['created_at'], user['url'])
                    cursor.execute("INSERT INTO user (id, name, screen_name, location, description, followers_count, "
                               "friends_count, favourites_count, statuses_count, verified, created_at, url) "
                               "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", user_values)
                    
                
                #adding quoted tweet
                if (data['is_quote_status']):
                    cursor.execute("SELECT * FROM user WHERE id = %s", (data['quoted_status']['user']['id_str'],))
                    if (cursor.fetchone() is None):
                        user = data['quoted_status']['user']
                        user_id = user['id_str']
                        user_values = (user_id, user['name'], user['screen_name'], user['location'], user['description'], 
                         user['followers_count'], user['friends_count'], user['favourites_count'], user['statuses_count'], 
                         user['verified'], user['created_at'], user['url'])
                        cursor.execute("INSERT INTO user (id, name, screen_name, location, description, followers_count, "
                                   "friends_count, favourites_count, statuses_count, verified, created_at, url) "
                                   "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", user_values)
                
                #adding retweeted tweet user details
                cursor.execute("SELECT * FROM user WHERE id = %s", (data['retweeted_status']['user']['id_str'],))
                if (cursor.fetchone() is None):
                    user = data['retweeted_status']['user']
                    user_id = user['id_str']
                    user_values = (user_id, user['name'], user['screen_name'], user['location'], user['description'], 
                     user['followers_count'], user['friends_count'], user['favourites_count'], user['statuses_count'], 
                     user['verified'], user['created_at'], user['url'])
                    cursor.execute("INSERT INTO user (id, name, screen_name, location, description, followers_count, "
                               "friends_count, favourites_count, statuses_count, verified, created_at, url) "
                               "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", user_values)
                
            else:
                 #adding quoted tweet
                if (data['is_quote_status']):    #check if tweet is quoted
                    cursor.execute("SELECT * FROM user WHERE id = %s", (data['quoted_status']['user']['id_str'],))
                    if (cursor.fetchone() is None):
                        user = data['quoted_status']['user']
                        user_id = user['id_str']
                        user_values = (user_id, user['name'], user['screen_name'], user['location'], user['description'], 
                         user['followers_count'], user['friends_count'], user['favourites_count'], user['statuses_count'], 
                         user['verified'], user['created_at'], user['url'])
                        cursor.execute("INSERT INTO user (id, name, screen_name, location, description, followers_count, "
                                   "friends_count, favourites_count, statuses_count, verified, created_at, url) "
                                   "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", user_values)
                
                #adding retweeted tweet user details
                cursor.execute("SELECT * FROM user WHERE id = %s", (data['user']['id_str'],))
                if (cursor.fetchone() is None):
                    user = data['user']
                    user_id = user['id_str']
                    user_values = (user_id, user['name'], user['screen_name'], user['location'], user['description'], 
                     user['followers_count'], user['friends_count'], user['favourites_count'], user['statuses_count'], 
                     user['verified'], user['created_at'], user['url'])
                    cursor.execute("INSERT INTO user (id, name, screen_name, location, description, followers_count, "
                               "friends_count, favourites_count, statuses_count, verified, created_at, url) "
                               "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", user_values)
        except:
            continue

In [10]:
cursor.execute("SELECT count(distinct(name)) FROM user")
users_count = cursor.fetchall()
users_count

[(18601,)]

In [6]:
import time

start_time = time.time()

cursor.execute("SELECT * FROM user LIMIT 20")
result = cursor.fetchall()

end_time = time.time()
total_time = end_time - start_time

print("Query took {:.2f} seconds".format(total_time))

Query took 0.01 seconds


In [13]:
import time

start_time = time.time()

cursor.execute("SELECT * FROM user")
result = cursor.fetchall()

end_time = time.time()
total_time = end_time - start_time

print("Query took {:.2f} seconds".format(total_time))

Query took 0.31 seconds


In [15]:
cursor.close()
cnx.close()